<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2">Functions</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-3">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-4">Main</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Get-aaFeatures-for-each-mixcrfile" data-toc-modified-id="Get-aaFeatures-for-each-mixcrfile-4.0.1">Get aaFeatures for each mixcrfile</a></span></li></ul></li></ul></li><li><span><a href="#plot" data-toc-modified-id="plot-5">plot</a></span></li><li><span><a href="#NMI" data-toc-modified-id="NMI-6">NMI</a></span></li></ul></div>

# Dependencies

In [1]:
pacman::p_load(igraph, graphlayouts, ggraph, ggforce, dplyr)

# Functions

In [2]:
#source("/Users/anabbi/git/ped_CapTCRseq/R/Immune_diversity.R")

In [1]:
aaFeatures.fx <- function(mixcrfle, aalength, outdir){
    
    aaT <- read.csv(file = "~/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures/aa_attributes.csv", header = T, row.names = 1)
    
    mysample <- gsub(".txt","", mixcrfle)
    mysample <- gsub(".*CLONES_TRB", "", mysample)
    
    f1 <- read.table(file = mixcrfle, sep = "\t", header = T, stringsAsFactors = F)
    
    message(paste("Number of CDR3s in:", mixcrfle))
    print(nrow(f1))
    
    #remove nonproductive
    f1 <- f1[ !grepl("[_]", f1$aaSeqCDR3),]
    f1 <- f1[ !grepl("[*]", f1$aaSeqCDR3),]
    
    message(paste("Number of CDR3s in:", mixcrfle, "nonproductive removed"))
    print(nrow(f1))
    
    f1$aa_length <- nchar(f1$aaSeqCDR3)
    
    f1_subset <- f1[ f1$aa_length == aalength,]
    
    if(nrow(f1_subset) <= 1){
        stop(paste0("less than one CDR3 at length:", aalength, "for file:", mixcrfle))
    }
    
    myaamat_list <- vector("list", aalength)

    myaamat <- matrix(nrow = 20, ncol = 6,
                      dimnames = list(rownames(aaT), 
                                      c("Freq","F1", "F2", "F3", "F4", "F5")))
    for( i in 1:aalength){
    mytab <- table(sapply(strsplit(f1_subset$aaSeqCDR3, ""),"[[",i) )
    myaamat[rownames(mytab),"Freq"] <- mytab
    myaamat[rownames(mytab), 2:6] <- unlist(aaT[rownames(mytab), 1:5]) * 
                                        myaamat[rownames(mytab),"Freq"]
    myaamat_list[[i]] <- myaamat
    myaamat[!is.na(myaamat)] <- NA
}
    
    save(myaamat_list, file = paste0(outdir, mysample, "_length", aalength,".RData"))
}

# Paths

In [2]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"

gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

In [3]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/"

# Main

In [31]:
aaT <- read.csv(file = paste0(datapath, "aa_attributes.csv"), header = T, row.names = 1)

In [32]:
#Table 2 from Atchley et al
head(aaT)

,Factor.I,Factor.II,Factor.III,Factor.IV,Factor.V
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A,-0.591,-1.302,-0.733,1.570,-0.146
C,-1.343,0.465,-0.862,-1.020,-0.255
D,1.050,0.302,-3.656,-0.259,-3.242
E,1.357,-1.453,1.477,0.113,-0.837
F,-1.006,-0.590,1.891,-0.397,0.412
G,-0.384,1.652,1.330,1.045,2.064


### Get aaFeatures for each mixcrfile


run locally

In [4]:
load(file = paste0(datapath,"capTCRseq/TRBfiles.RData"))

In [5]:
myfilelist <- paste0("~/Desktop/H4H/INTERCEPT/NextSeq/", file_df_dedup_trb$file_list, sep = "")

In [6]:
head(myfilelist)

[1] "~/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt"     
[2] "~/Desktop/H4H/INTERCEPT/NextSeq/batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-01-PBMC-DNA.txt"
[3] "~/Desktop/H4H/INTERCEPT/NextSeq/batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-02-PBMC-DNA.txt"
[4] "~/Desktop/H4H/INTERCEPT/NextSeq/batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-03-PBMC-DNA.txt"
[5] "~/Desktop/H4H/INTERCEPT/NextSeq/batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-04-PBMC-DNA.txt"
[6] "~/Desktop/H4H/INTERCEPT/NextSeq/batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_353-02-PBMC-DNA.txt"

In [12]:
length15 <- lapply(myfilelist, function(x){ 
    tryCatch(
    aaFeatures.fx(x, 15, "~/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures/"), error = function(e) e
    )
})

Number of CDR3s in: ~/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt



[1] 39


Number of CDR3s in: ~/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt nonproductive removed



[1] 27


Number of CDR3s in: ~/Desktop/H4H/INTERCEPT/NextSeq/batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-01-PBMC-DNA.txt



...

[1] 0


Number of CDR3s in: ~/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRBCHP_383-03_cfDNA.txt



[1] 3


Number of CDR3s in: ~/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRBCHP_383-03_cfDNA.txt nonproductive removed



[1] 2


In [13]:
length14 <- lapply(myfilelist, function(x){ 
    tryCatch(
    aaFeatures.fx(x, 14, "~/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures/"), error = function(e) e
    )
})

Number of CDR3s in: ~/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt



[1] 39


Number of CDR3s in: ~/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt nonproductive removed



[1] 27


Number of CDR3s in: ~/Desktop/H4H/INTERCEPT/NextSeq/batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-01-PBMC-DNA.txt



...

[1] 0


Number of CDR3s in: ~/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRBCHP_383-03_cfDNA.txt



[1] 3


Number of CDR3s in: ~/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRBCHP_383-03_cfDNA.txt nonproductive removed



[1] 2


In [ ]:
length16 <- lapply(myfilelist, function(x){ 
    tryCatch(
    aaFeatures.fx(x, 16, "~/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures/"), error = function(e) e
    )
})

Number of CDR3s in: ~/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt



[1] 39


Number of CDR3s in: ~/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt nonproductive removed



[1] 27


Number of CDR3s in: ~/Desktop/H4H/INTERCEPT/NextSeq/batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-01-PBMC-DNA.txt



...

Number of CDR3s in: ~/Desktop/H4H/INTERCEPT/NextSeq/batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-01-PBMC-DNA.txt nonproductive removed



[1] 153


Number of CDR3s in: ~/Desktop/H4H/INTERCEPT/NextSeq/batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-02-PBMC-DNA.txt



[1] 362


Number of CDR3s in: ~/Desktop/H4H/INTERCEPT/NextSeq/batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-02-PBMC-DNA.txt nonproductive removed



In [ ]:
length13 <- lapply(myfilelist, function(x){ 
    tryCatch(
    aaFeatures.fx(x, 13, "~/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures/"), error = function(e) e
    )
})

# plot

In [9]:
load(paste0(h4hpath, "aaFeatures/CHP_358-02-PBMC-DNA_length15.RData"))

In [205]:
sum_features_aaP <- lapply(myaamat_list, function(x){ colSums(x[,2:6], na.rm = T)})

In [207]:
sum_features_aaP

[[1]]
     F1      F2      F3      F4      F5 
-10.744   3.720  -6.896  -8.160  -2.040 

[[2]]
     F1      F2      F3      F4      F5 
 -4.728 -10.416  -5.864  12.560  -1.168 

[[3]]
     F1      F2      F3      F4      F5 
 -1.824  11.192 -38.080   5.360 -21.176 

[[4]]
     F1      F2      F3      F4      F5 
 -1.628  10.119 -31.107   5.598 -17.216 

[[5]]
    F1     F2     F3     F4     F5 
-4.020 -1.098 -4.704  3.723 -5.319 

[[6]]
    F1     F2     F3     F4     F5 
-1.641  2.804  8.853  2.802  6.363 

[[7]]
    F1     F2     F3     F4     F5 
-1.992  5.916  5.914  8.501 12.159 

[[8]]
    F1     F2     F3     F4     F5 
-0.070  3.896  0.465  6.168  3.685 

[[9]]
    F1     F2     F3     F4     F5 
-0.050  5.755 -0.396  4.063 -0.284 

[[10]]
    F1     F2     F3     F4     F5 
-1.262  6.177 -5.539  5.904 -0.708 

[[11]]
    F1     F2     F3     F4     F5 
 4.481  2.178 -3.613  1.208 -6.452 

[[12]]
   F1    F2    F3    F4    F5 
4.827 1.673 6.870 3.515 1.129 

[[13]]
     F1      F2      F3      F4      F5 
 -2.094  -5.347 -12.132   4.252  -7.625 

[[14]]
    F1     F2     F3     F4     F5 
-0.300  1.306 12.824 -7.094  6.716 

[[15]]
    F1     F2     F3     F4     F5 
-8.048 -4.720 15.128 -3.176  3.296

# NMI

In [28]:
mat <- sapply(as.character(c("CASSFYGREADTQYF", "CASSVVGVYNSPLHF", "CASSLTRPSGNTIYF",
                            "CASSYQAGVGETQYF")), function(x){
      x <- as.character(x);
      x <- strsplit(x, "");
      x <- unlist( x )
   })

mat <- t(mat)

In [29]:
 res <- data.frame()
   for( i in 1:15 ){
   for( k in 1:15 ){
      print(paste0( i, ":", k))
      d1 <- mat[, c(i,k)]
      d1 <- as.data.frame(d1)
      colnames(d1) <- c("X","Y")
      d1 <- d1[ d1$X!="NA" &  d1$Y!="NA",] #revemo NA
      
      xlist <- as.character( unique(d1$X) )
      ylist <- as.character( unique(d1$Y) )
      
      #make frequency table
      freq <- matrix(0, nrow=length(xlist), ncol=length(ylist))
      row.names(freq) <- xlist
      colnames(freq) <- ylist
      
      for( x in xlist ){
      for( y in ylist ){
         both <- subset(d1, X==x & Y==y)
         freq[ x, y ] <- nrow( both )
      }
      }
      
      MI <- mi.empirical(freq, unit=c("log"))
      Hx <- entropy.empirical( colSums(freq) )
      Hy <- entropy.empirical( rowSums(freq) )
      NMI <- 2 * MI / ( Hx + Hy ) #normalized mutual entropy
      
      dump <- data.frame(i, k, MI, Hx, Hy, NMI)
      res <- rbind(res, dump)
      
   }
   }

[1] "1:1"
[1] "1:2"
[1] "1:3"
[1] "1:4"
[1] "1:5"
[1] "1:6"
[1] "1:7"
[1] "1:8"
[1] "1:9"
[1] "1:10"
[1] "1:11"
[1] "1:12"
[1] "1:13"
[1] "1:14"
[1] "1:15"
[1] "2:1"
[1] "2:2"
[1] "2:3"
[1] "2:4"
[1] "2:5"
[1] "2:6"
[1] "2:7"
[1] "2:8"
[1] "2:9"
[1] "2:10"
[1] "2:11"
[1] "2:12"
[1] "2:13"
[1] "2:14"
[1] "2:15"
[1] "3:1"
[1] "3:2"
[1] "3:3"
[1] "3:4"
[1] "3:5"
[1] "3:6"
[1] "3:7"
[1] "3:8"
[1] "3:9"
[1] "3:10"
[1] "3:11"
[1] "3:12"
[1] "3:13"
[1] "3:14"
[1] "3:15"
[1] "4:1"
[1] "4:2"
[1] "4:3"
[1] "4:4"
[1] "4:5"
[1] "4:6"
[1] "4:7"
[1] "4:8"
[1] "4:9"
[1] "4:10"
[1] "4:11"
[1] "4:12"
[1] "4:13"
[1] "4:14"
[1] "4:15"
[1] "5:1"
[1] "5:2"
[1] "5:3"
[1] "5:4"
[1] "5:5"
[1] "5:6"
[1] "5:7"
[1] "5:8"
[1] "5:9"
[1] "5:10"
[1] "5:11"
[1] "5:12"
[1] "5:13"
[1] "5:14"
[1] "5:15"
[1] "6:1"
[1] "6:2"
[1] "6:3"
[1] "6:4"
[1] "6:5"
[1] "6:6"
[1] "6:7"
[1] "6:8"
[1] "6:9"
[1] "6:10"
[1] "6:11"
[1] "6:12"
[1] "6:13"
[1] "6:14"
[1] "6:15"
[1] "7:1"
[1] "7:2"
[1] "7:3"
[1] "7:4"
[1] "7:5"
[1] "7:6"
[1] 

In [34]:
res

i,k,MI,Hx,Hy,NMI
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0,0.0000000,0,NaN
1,2,0,0.0000000,0,NaN
1,3,0,0.0000000,0,NaN
1,4,0,0.0000000,0,NaN
1,5,0,1.3862944,0,0
1,6,0,1.3862944,0,0
1,7,0,1.0397208,0,0
1,8,0,1.3862944,0,0
1,9,0,1.3862944,0,0
